GRU模型
===

GRU(Gated Recurrent Unit)也称作门控循环单元结构，他也是传统RNN的变体，通LSTM一样能够有效捕捉长序列之间的语义关联，缓解梯度消失或爆炸现象，同时它的结构和计算要比LSTM更简单，它的核心结构可以分为两个部分去解析：更新门、重置门

# 1.GRU的内部结构图与计算公式

![images](images/020.png)

$$
\begin{cases}
z_t&=&\sigma[W_z \bullet (h_{t-1},x_t)] \\
r_t&=&\sigma[W_r \bullet (h_{t-1},x_t)] \\
\tilde{h_t}&=&tanh[W \bullet (r_t \times h_{t-1}, x_t)] \\
h_t&=&(1-z_t) \times h_{t-1} + z_t \times \tilde{h_t}
\end{cases}
$$

$z_t$是更新门，$r_t$就是重置门

- 计算更新门和重置门的门值，分别是$z_t$和$r_t$，计算方法就是使用$X_t$与$h_{t-1}$拼接进行线性变换在经过sigmoid激活
- 更新门门值的左右在了$h_{t-1}$上，代表控制上一时间步传来的信息有多少可以被利用
- 使用这个更新后的$h_{t-1}$进行基本的RNN计算，即与$x_t$拼接进行线性变换，经过tanh激活，得到新的$h_t$
- 重置门的门值会作用在新的$h_t$，而1减去门值会作用在$h_{t-1}$，随后将两者的结果相加，得到最终的隐含状态输出$h_t$
- 这个过程意味着重置门有能力重置之前所有的计算，当门值趋于1时，输出就是新的$h_t$，而当门值趋于0时，输出就是上一时间步的$h_{t-1}$

Bi-GRU和Bi-LSTM的逻辑相同，都是不改变其内部结构，而是将模型应用两次且方向不同，再将两次得到的LSTM结果进行拼接作为最终输出。

# 2.GRU在Pytorch中的实现

In [1]:
import torch
import torch.nn as nn
rnn = nn.GRU(5, 6, 2)
input = torch.randn(1, 3, 5)
h0 = torch.randn(2, 3, 6)
output, hn = rnn(input, h0)
output

tensor([[[ 0.2275, -0.9000, -0.6166,  0.4027,  0.5367, -0.0312],
         [ 0.9569,  0.1598, -0.4007,  0.1350,  0.5063, -0.6303],
         [-0.3321, -0.1651, -0.1089, -0.3715,  0.6901,  0.1050]]],
       grad_fn=<StackBackward>)

In [2]:
hn

tensor([[[-0.9484, -0.6072,  0.0513,  1.4981,  0.4362,  0.3444],
         [ 0.4520,  0.0805, -0.9564, -0.3014, -0.6984, -0.2426],
         [-0.2173,  0.2282,  0.5268,  0.3826,  0.0648,  0.4860]],

        [[ 0.2275, -0.9000, -0.6166,  0.4027,  0.5367, -0.0312],
         [ 0.9569,  0.1598, -0.4007,  0.1350,  0.5063, -0.6303],
         [-0.3321, -0.1651, -0.1089, -0.3715,  0.6901,  0.1050]]],
       grad_fn=<StackBackward>)

# 3.GRU的优势和劣势

## 3.1.优势
GRU和LSTM作用相同，在捕捉长序列语义关联时，能有效抑制梯度消失或爆炸，效果都优于传统RNN且计算复杂度相比LSTM要小

## 3.2.缺点
GRU仍然不能完全解决梯度消失问题，同时其作用RNN的变体，有着RNN结构本身的一大弊端，即不可并行计算，这在数据量和模型体量逐步增大的未来，是RNN发展的关键瓶颈